# Filtado de mensajes spam

## Descripción del problema real

La recepción de publicidad no deseada a traves mensajes de texto usando SMS (Short Message Service) es un problema que afecta a muchos usuarios de teléfonos móviles. El problema radica en que los usuarios deben pagar por los mesajes recibidos, y por este motivo resulta muy importante que las compañías prestadoras del servicio puedan filtrar mensajes indeseados antes de enviarlos a su destinatario final. Los mensajes tienen una longitud máxima de 160 caracteres, por lo que el texto resulta poco para realizar la clasificación, en comparación con textos más largos (como los emails). Adicionalmente, los errores de digitación dificultan el proceso de detección automática.

## Descripción del problema en términos de los datos

Se tiene una muestra contiene 5574 mensajes en inglés, no codificados y clasificados como legítimos (ham) o spam (http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). La información está almacenada en el archivo `datos/spam-sms.zip`.El problema en términos de los datos consiste en clasificar si un mensaje SMS es legítico o spam, a partir del análisis de las palabras que contiente, partiendo del supuesto de que ciertas palabras que son más frecuentes dependiendo del tipo de mensaje. Esto implica que en la fase de preparación de los datos se deben extraer las palabras que contiene cada mensaje para poder realizar el análsis.

## Aproximaciones posibles

En este caso, se desea comparar los resultados de un modelo de redes neuronales artificiales y otras técnicas estadísticas para realizar la clasificación.

## Requerimientos

Usted debe:

* Preprocesar los datos para representarlos usando bag-of-words.


* Construir un modelo de regresión logística como punto base para la comparación con otros modelos más complejos.


* Construir un modelo de redes neuronales artificiales. Asimismo, debe determinar el número de neuronas en la capa o capas ocultas.


* Utiizar una técnica como crossvalidation u otra similar para establecer la robustez del modelo.


* Presentar métricas de desempeño para establecer las bondades y falencias de cada clasificador.

## Angel Ricardo Racini Meza
### C.C 1017248189

In [59]:
#Importando librerias
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

In [7]:
#Leyendo el dataset
df = pd.read_csv(r"C:\Users\ANGEL\Desktop\Trabajo Redes Neuronales\evaluacion-arracinim\SMSSpamCollection.txt", sep = "\t", header=None)

In [ ]:
#Se descargan las librerias necesarias para procesar el texto 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

## Preprocesando los datos 

Para este ejercicio, se usará la libreria NLTK de Python. Con el fin de dividir cada frase en tokens semanticos de con los que se pueda sacar estadisticos como el promedio de verbos por mensajes, promedio de simbolos por mensajes, etc.

Para esto se tiene que las palabras se pueden dividir en Articulos, Sustantivo, Pronombre, Adjetivo, verbo y adverbio. Además, para este caso se tendrán en cuenta tambien la cantidad de simbolos y numeros y la longitud total del texto.

In [80]:
# Diviendo el texto en una lista de tokens, clasificanco y sacando los promedios. 
lista = []
for index, row in df.iterrows():
    longitud = 0; verbo = 0; preposicion = 0; sustantivo = 0; pronombre = 0; adjetivo = 0; adverbio = 0
    numero = 0; simbolo = 0; lista_tokenizada = []; Y = None

    Y = row[0]
    
    #filtramos las stopwords o en español palabras vacias. 
    stop_words = set(stopwords.words('english')) 
    word_tokens = nltk.word_tokenize(row[1]) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 

    #Clasificamos el resto en las clasificaciones dadas
    words = nltk.pos_tag(filtered_sentence)
    longitud = len(words)

    for word in words:
        if word[1] == "CD":
            numero += 1
        elif word[1] == "IN" or word[1] == "TO":
            preposicion += 1
        elif word[1] == "JJ" or word[1] == "JJR" or word[1] == "JJS" or word[1] == "JJ":
            adjetivo += 1
        elif word[1] == "NN" or word[1] == "NNP" or word[1] == "NNS":
            sustantivo += 1
        elif word[1] == "PRP" or word[1] == "PRP$":
            pronombre += 1
        elif word[1] == "RB" or word[1] == "RBR" or word[1] == "RBS" or word[1] == "JJ":
            adverbio += 1
        elif word[1] == "VB" or word[1] == "VBD" or word[1] == "VBG" or word[1] == "VBN" or word[1] == "VBP" or word[1] == "VBZ":
            verbo += 1
        else:
            simbolo += 1 
        

    lista_tokenizada = [longitud , verbo,  preposicion, pronombre, adjetivo , adverbio, 
    sustantivo, numero, simbolo, Y]
    lista.append(lista_tokenizada)

df_final = pd.DataFrame(data = lista, columns = ['longitud' , 'verbo',  'preposicion', 'pronombre', 'adjetivo' , 'adverbio', 'sustantivo', 'numero', 'simbolo', 'Y'])

In [81]:
#Vemos la estructura del nuevo dataframe 
df_final.head()

,longitud,verbo,preposicion,pronombre,adjetivo,adverbio,sustantivo,numero,simbolo,Y
0,19,2,0,0,2,2,9,0,4,ham
1,8,0,0,0,1,0,4,0,3,ham
2,32,2,0,0,5,0,14,5,6,spam
3,11,3,0,0,2,1,3,0,2,ham
4,10,3,2,1,0,2,1,0,1,ham


In [82]:
# Se procede a convertir el target a una variable binaria. 
df_final['Y'] = df_final['Y'].apply(lambda x: 1 if x == "spam" else 0)
df_final.head()

,longitud,verbo,preposicion,pronombre,adjetivo,adverbio,sustantivo,numero,simbolo,Y
0,19,2,0,0,2,2,9,0,4,0
1,8,0,0,0,1,0,4,0,3,0
2,32,2,0,0,5,0,14,5,6,1
3,11,3,0,0,2,1,3,0,2,0
4,10,3,2,1,0,2,1,0,1,0


## Analisis descriptivo de las variables

